In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pickle
%matplotlib inline

In [61]:
df = pd.read_pickle("final_df.pkl")

In [62]:
df.drop([
 'ticker',
 'date',
 'calendardate',
 'name',
 'rating_cnt_strong_buys',
 'rating_cnt_mod_buys',
 'rating_cnt_holds',
 'rating_cnt_mod_sells',
 'rating_cnt_strong_sells',
 'rating_cnt_with',
 'rating_cnt_without',
 'rating_change',
 'quart',
 'year'
 ], axis=1, inplace=True)

In [63]:
one_hot_features = ['exchange', 'sector', 'industry']

In [64]:
for feature in one_hot_features:
    cat_X = df.loc[:, [feature]]
    ohe = OneHotEncoder(drop='first', sparse=False)
    ohe.fit(cat_X)
    ohe_X = ohe.transform(cat_X)
    columns = ohe.get_feature_names([feature])
    ohe_X_df = pd.DataFrame(ohe_X, columns=columns, index=cat_X.index)
    df = df.join(ohe_X_df)

In [65]:
df.drop([
 'exchange',
 'sector',
 'industry'   
 ], axis=1, inplace=True)

In [66]:
df.info(verbose=True, null_counts=True)

<ipython-input-66-6ac79f7ef903>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081593 entries, 0 to 1081592
Data columns (total 325 columns):
 #    Column                                             Non-Null Count    Dtype  
---   ------                                             --------------    -----  
 0    close                                              1081593 non-null  float64
 1    volume                                             1081593 non-null  float64
 2    accoci                                             1081593 non-null  float64
 3    assets                                             1081593 non-null  float64
 4    assetsavg                                          1081593 non-null  float64
 5    assetsc                                            1081593 non-null  float64
 6    assetsnc                                           1081593 non-null  float64
 7    assetturnover                                      1081593 non-null  float64
 8    bvps                                              

In [67]:
df

,close,volume,accoci,assets,assetsavg,assetsc,assetsnc,assetturnover,bvps,capex,...,industry_Travel Services,industry_Trucking,industry_Uranium,industry_Utilities - Diversified,industry_Utilities - Independent Power Producers,industry_Utilities - Regulated Electric,industry_Utilities - Regulated Gas,industry_Utilities - Regulated Water,industry_Utilities - Renewable,industry_Waste Management
0,78.56,2076523.0,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,0.576,15.836,-155000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,77.89,3570828.0,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,0.576,15.836,-155000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,78.42,2797669.0,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,0.576,15.836,-155000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,79.34,1878041.0,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,0.576,15.836,-155000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,78.55,1992595.0,-381000000.0,8.952000e+09,8.656500e+09,3.712000e+09,5.240000e+09,0.576,15.836,-155000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081588,18.07,646106.0,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,1.547,1.679,-985000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081589,18.42,315802.0,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,1.547,1.679,-985000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081590,17.42,388872.0,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,1.547,1.679,-985000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1081591,14.52,2337972.0,0.0,7.217500e+07,5.178025e+07,6.302300e+07,9.152000e+06,1.547,1.679,-985000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
col_list = list(df.columns)

In [69]:
for col in col_list:
    print(col, np.isinf(df[col]).values.sum())

close 0
volume 0
accoci 0
assets 0
assetsavg 0
assetsc 0
assetsnc 0
assetturnover 0
bvps 0
capex 0
cashneq 0
cor 0
consolinc 0
currentratio 0
de 0
debt 0
debtc 0
debtnc 0
deferredrev 0
depamor 0
deposits 0
dps 0
ebit 0
ebitda 0
ebitdamargin 0
ebt 0
eps 0
epsdil 0
equity 0
equityavg 0
fcf 0
fcfps 0
gp 0
grossmargin 0
intangibles 0
intexp 0
invcap 0
invcapavg 0
inventory 0
investments 0
investmentsc 0
investmentsnc 0
liabilities 0
liabilitiesc 0
liabilitiesnc 0
ncf 0
ncfbus 0
ncfcommon 0
ncfdebt 0
ncfdiv 0
ncff 0
ncfi 0
ncfinv 0
ncfo 0
ncfx 0
netinc 0
netinccmn 0
netincdis 0
netincnci 0
netmargin 0
opex 0
opinc 0
payables 0
payoutratio 0
ppnenet 0
prefdivis 0
ps 0
ps1 0
receivables 0
retearn 0
revenue 0
rnd 0
roa 0
roe 0
roic 0
ros 0
sbcomp 0
sgna 0
sharefactor 0
sharesbas 0
shareswa 0
shareswadil 0
sps 0
tangibles 0
taxassets 0
taxexp 0
taxliabilities 0
tbvps 0
workingcapital 0
revenue_1y_pct 0
gp_1y_pct 0
ebitda_1y_pct 0
opinc_1y_pct 0
netinc_1y_pct 0
eps_1y_pct 0
assets_1y_pct 0
ncfo_

In [70]:
df_X = df.copy()

In [71]:
del df_X["rating_mean_recom"]

In [72]:
X = df_X

y = df["rating_mean_recom"]

In [73]:
#Split the data 80-20 train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=15)

In [74]:
std = StandardScaler()
std.fit(X_train.values)
X_tr = std.transform(X_train.values)
X_te = std.transform(X_test.values)
lasso_model = LassoCV(cv=5, max_iter = 1000000)
X_train_model = lasso_model.fit(X_tr, y_train)
X_train_model.score(X_tr, y_train)

0.12589608580448042

In [75]:
lasso_model.alpha_

0.0001414896381593165

In [76]:
coef = list(zip(X_train.columns, lasso_model.coef_))
coef

[('close', -0.0),
 ('volume', 0.003973596705814853),
 ('accoci', 0.030607010074823908),
 ('assets', -0.0),
 ('assetsavg', -0.0),
 ('assetsc', -0.0),
 ('assetsnc', 0.0),
 ('assetturnover', 0.014901890719409033),
 ('bvps', -0.024061817473467192),
 ('capex', -0.013839161126220363),
 ('cashneq', 0.053286377232147214),
 ('cor', -0.04436723016835459),
 ('consolinc', -0.023144164101311506),
 ('currentratio', 0.0034180262558260606),
 ('de', 3.6690128183790035e-05),
 ('debt', 0.049271863096821904),
 ('debtc', 0.03519251221316122),
 ('debtnc', -0.0452256690810431),
 ('deferredrev', -0.002476321398633969),
 ('depamor', 0.04813033935294777),
 ('deposits', 0.007007057120279524),
 ('dps', -0.009450279929087157),
 ('ebit', -0.0),
 ('ebitda', 0.0),
 ('ebitdamargin', 0.4992135684860955),
 ('ebt', -0.0),
 ('eps', -0.0),
 ('epsdil', -0.043668029680954595),
 ('equity', -0.007930862308645158),
 ('equityavg', 0.0),
 ('fcf', -0.0236180913683607),
 ('fcfps', 0.0594615359284992),
 ('gp', -0.0),
 ('grossmargin'

In [77]:
test_set_pred = lasso_model.predict(X_te)

In [78]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

In [79]:
mae(y_test, test_set_pred)

0.7278933014722484

In [80]:
r2_score(y_test, test_set_pred)

0.12512977653090196

In [83]:
filename = 'lasso_model.pkl'
pickle.dump(lasso_model, open(filename, 'wb'))